<p style="padding: 10px;
          color: #07663c;
          text-align: center;
          font-size:170%;">
Spark ile Sınırlandırma (Classification on PySpark)
</p>

- [<span style="font-size:130%; color:#07663c; ">Kütüphaneler (Import Libraries)</span>](#1)
- [<span style="font-size:130%; color:#07663c; ">Veri Yükleme (Reading Data)</span>](#2)
- [<span style="font-size:130%; color:#07663c; ">Veri Seti Kontrolü (Checking Dataset)</span>](#3)
- [<span style="font-size:130%; color:#07663c; ">Eksik veriler (Missing Values)</span>](#4)
- [<span style="font-size:130%; color:#07663c; ">Değişkenlerin Sınıflandırılması (Feature Classification)</span>](#5)
- [<span style="font-size:130%; color:#07663c; ">Hedef Değişken Kontrolü (Target Analysis)</span>](#6)
- [<span style="font-size:130%; color:#07663c; ">Kategorik Değişken Analizi (Categorical Variable Analysis)</span>](#7)
- [<span style="font-size:130%; color:#07663c; ">Veri Ön İşleme (Data Preprocessing)</span>](#8)
- [<span style="font-size:130%; color:#07663c; ">Modelleme (Modeling)</span>](#9)
- [<span style="font-size:130%; color:#07663c; ">Değerlendirme (Evalution)</span>](#10)

<a id = "1"></a> <span style="font-size:150%; color:#07663c; ">Kütüphaneler (Import Libraries)</span>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=7a04ebd37ca0739e5baaf2f4dbeb7c803bbb9879e6e48571a33c555707e694d5
  Stored in directory: /root/.cache/pip/wheels/07/fb/67/b9f2c0242d156eaa136b45ae4fd99d3e7c0ecc2acfd26f47b9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>

In [4]:
from pyspark.sql import SparkSession # Dataframe oluşturmak için gerekli.
spark = SparkSession.builder.appName("Basics").getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 13:39:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm

In [6]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<a id = "2"></a> <span style="font-size:150%; color:#07663c; ">Veri Yükleme (Reading Data)</span>

In [8]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/churn-modelling-bank-customer/Churn_Modelling.csv


In [9]:
df = (spark.read.format("csv")
      .option("header",True)
      .option("inferSchema",True)
      .load("/kaggle/input/churn-modelling-bank-customer/Churn_Modelling.csv")
     )

<a id = "3"></a> <span style="font-size:150%; color:#07663c; ">Veri Seti Kontrolü (Checking Dataset)</span>

In [10]:
print(df.count())
# Veri setindeki satır sayısını verir.

10000


In [11]:
print(len(df.columns))
# Veri setindeki değişken (sütun) sayısını verir.

14


* Verisetinde; 
    * 10.000 satır,
    * 14 sütun (değişken) bulunuyor.

In [12]:
df.limit(15).toPandas()
# ilk 15 gözleme göz gezdiriyoruz.

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [13]:
df.printSchema()
# Veri tipi ve boş değer kontrolü.

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



* Veri tipinde herhangi bir dönüşüme ihtiyaç bulunmamaktadır.

In [14]:
df.persist()
# Veri küçük olduğu için Persist ediyoruz. Ek olarak veri tipi kontrolü yapabilirim.

DataFrame[RowNumber: int, CustomerId: int, Surname: string, CreditScore: int, Geography: string, Gender: string, Age: int, Tenure: int, Balance: double, NumOfProducts: int, HasCrCard: int, IsActiveMember: int, EstimatedSalary: double, Exited: int]

In [15]:
df.dtypes
# Veri tipi kontrolü.

[('RowNumber', 'int'),
 ('CustomerId', 'int'),
 ('Surname', 'string'),
 ('CreditScore', 'int'),
 ('Geography', 'string'),
 ('Gender', 'string'),
 ('Age', 'int'),
 ('Tenure', 'int'),
 ('Balance', 'double'),
 ('NumOfProducts', 'int'),
 ('HasCrCard', 'int'),
 ('IsActiveMember', 'int'),
 ('EstimatedSalary', 'double'),
 ('Exited', 'int')]

In [16]:
df_count = df.count()
print(df_count)

10000


<a id = "4"></a> <span style="font-size:150%; color:#07663c; ">Eksik Veriler (Missing Values)</span>

In [17]:
for col_name in df.dtypes:
    null_count = df.filter( (F.col(col_name[0]).isNull()) | (F.col(col_name[0]) == "")).count()
    
    if null_count > 0:
        print("{}  {} type null values: {} % {}".format(col_name[0], col_name[1], null_count, (null_count/df_count * 100)))

* Eksik gözlem bulunmamaktadır.

<a id = "5"></a> <span style="font-size:150%; color:#07663c; ">Değişkenlerin Sınıflandırılması (Feature Classification)</span>

#### Değişkenleri Kategorik, Numeric, Hedef ve Çıkarılacak olarak sınıflandırıyoruz.

In [18]:
categoric_cols = []

numeric_cols = []

discarted_cols = ['RowNumber', 'CustomerId', 'Surname']

label_col = ['Exited']

In [19]:
for col_name in df.dtypes:
    if (col_name[0] not in discarted_cols+label_col):
        if col_name[1] == 'string':
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])
            
# sınıflarına göre listelere atama işlemi yapan döngü.

In [20]:
print(categoric_cols)
print(len(categoric_cols))

['Geography', 'Gender']
2


In [21]:
print(numeric_cols )
print(len(numeric_cols))


['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
8


In [22]:
print(discarted_cols )
print(len(discarted_cols))

['RowNumber', 'CustomerId', 'Surname']
3


In [23]:
print(label_col)
print(len(label_col))

['Exited']
1


In [24]:
for col_name in categoric_cols:
    df1 = df.withColumn(col_name, F.trim(col_name))

<a id = "6"></a> <span style="font-size:150%; color:#07663c; ">Hedef Değişken Kontrolü (Target Analysis)</span>

In [25]:
df.select(label_col[0]).groupBy(label_col[0]).count().show()

+------+-----+
|Exited|count|
+------+-----+
|     1| 2037|
|     0| 7963|
+------+-----+



<a id = "7"></a> <span style="font-size:150%; color:#07663c; ">Kategorik Değişken Analizi (Categorical Variable Analysis)</span>

In [26]:
for col_name in categoric_cols:
    df.select(col_name).groupBy(col_name).count().show()


+---------+-----+
|Geography|count|
+---------+-----+
|  Germany| 2509|
|   France| 5014|
|    Spain| 2477|
+---------+-----+

+------+-----+
|Gender|count|
+------+-----+
|Female| 4543|
|  Male| 5457|
+------+-----+



#### Değişkenlerin kontrolü.

In [27]:
if len(df1.columns) == (len(categoric_cols)+len(numeric_cols)+len(discarted_cols)+len(label_col)):
    print("Değişkenlerin sayısı birbirine eşittir.")
else: print("Değişken gruplamada problem bulunmaktadır !")

Değişkenlerin sayısı birbirine eşittir.


<a id = "8"></a> <span style="font-size:150%; color:#07663c; ">Veri Ön İşleme (Data Preprocessing)</span>

## Encoding

In [28]:
tobeoheded = []
for col_name in categoric_cols:
    count = df1.select(col_name).distinct().count()

    if count > 2:
        tobeoheded.append(col_name)

In [29]:
print(tobeoheded)

['Geography']


## StringIndexter

In [30]:
from pyspark.ml.feature import StringIndexer

In [31]:
mydict = {}

stringindexerobj = []

stringindexeroutputnames = []

oheinputnames = []

oheoutputnames = []

for col_name in categoric_cols:
    mydict[col_name+"_indexedobj"] = StringIndexer() \
                                    .setHandleInvalid("skip") \
                                    .setInputCol(col_name) \
                                    .setOutputCol(col_name+"_indexed")
    stringindexerobj.append(mydict.get(col_name+"_indexedobj"))
    stringindexeroutputnames.append(col_name+"_indexed")

    if col_name in tobeoheded:
        oheinputnames.append(col_name+"_indexed")
        oheoutputnames.append(col_name+"_ohe")

In [32]:
print(stringindexerobj)

[StringIndexer_7c44c4a05dc4, StringIndexer_f30b08f28288]


In [33]:
print(stringindexeroutputnames)

['Geography_indexed', 'Gender_indexed']


In [34]:
print(oheinputnames)

['Geography_indexed']


In [35]:
print(oheoutputnames)

['Geography_ohe']


## OneHotEncoder İşlemi

In [36]:
from pyspark.ml.feature import OneHotEncoder

In [37]:
stringindexeroheexcluded = list(set(stringindexeroutputnames).difference(set(oheinputnames)))
print(stringindexeroheexcluded)

['Gender_indexed']


In [38]:
encoder = OneHotEncoder().setInputCols(oheinputnames).setOutputCols(oheoutputnames)

### Vector Assambler

In [39]:
from pyspark.ml.feature import VectorAssembler

In [40]:
assembler = VectorAssembler().setInputCols(numeric_cols+stringindexeroheexcluded+oheoutputnames) \
.setOutputCol("unscaled_features")

### Scaler

In [41]:
from pyspark.ml.feature import StandardScaler

In [42]:
scaler = StandardScaler().setInputCol("unscaled_features").setOutputCol("features")

<a id = "9"></a> <span style="font-size:150%; color:#07663c; ">Modelleme (Modeling)</span>

In [43]:
from pyspark.ml.classification import GBTClassifier

In [44]:
estimator = GBTClassifier() \
.setFeaturesCol("features") \
.setLabelCol(label_col[0])

### Pipeline

In [45]:
 from pyspark.ml import Pipeline

In [46]:
pipeline_obj = Pipeline().setStages(stringindexerobj+[encoder, assembler, scaler, estimator])

In [47]:
train_df, test_df = df1.randomSplit([.8, .2], seed = 142)

In [48]:
print(train_df.count())

7994


In [49]:
pipeline_model =  pipeline_obj.fit(train_df)

### Prediction

In [50]:
transformed_df = pipeline_model.transform(test_df)

In [51]:
transformed_df.select("Exited", "prediction").show(10)

+------+----------+
|Exited|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       1.0|
|     0|       0.0|
|     0|       0.0|
|     1|       0.0|
|     0|       0.0|
+------+----------+
only showing top 10 rows



<a id = "10"></a> <span style="font-size:150%; color:#07663c; ">Değerlendirme (Evalution)</span>

In [52]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [53]:
evaluator = BinaryClassificationEvaluator(labelCol = label_col[0])

In [54]:
evaluator.evaluate(transformed_df)

0.8591799680023813

In [55]:
evaluator.getMetricName()

'areaUnderROC'